# Extração de registros brasileiros do conjunto de dados GeoCoV19

## Resumo do Artigo GeoCoV19

O artigo GeoCov19 disponibiliza uma base de dados contendo mais de 524 milhões de registros de tweets, em 62 línguas, que fazem referência à Covid-19. Deste total, 94% do conjunto de dados estão geolocalizados.

Para a inclusão de informações de geolocalização foram consideradas informações de localização providas pelo próprio Twitter (com diferentes níveis de confiabilidade) e localizações extraídas a partir dos textos dos tweets (topônimos). A extração a partir dos textos considera qualquer menção a algum local como uma informação de localização válida.

Em atendimento à políticas do Twitter, os dados disponibilizados não possuem os textos dos tweets. O autor disponibiliza uma ferramenta para obtenção do conteúdo integral dos tweets através de seus "ids" (que são disponibilizados na base de dados GeoCov19).

Os dados coletados tem por objetivo capacitar comunidades de pesquisa a avaliar como as sociedades estão lidando coletivamente com a crise do Covid-19, desenvolver métodos para identificar notícias falsas, entender lacunas de conhecimento, contruir modelos de previsão, desenvolver alertas à doenças, entre outros.

## Objetivo da Atividade

Extrair dos dados de tweets citados acima, registros que fazem referência ao Brasil.

## Informações Técnicas

### Estrutura do arquivo JSON contendo tweets e informações de localização

1. **tweet_id**: it represents the Twitter provided id of a tweet

2. **created_at**: it represents the Twitter provided "created_at" date and time in UTC 

3. **user_id**: it represents the Twitter provided user id

4. **geo_source**: this field shows one of the four values: (i) coordinates, (ii) place, (iii) user_location, or (iv) tweet_text. The value depends on the availability of these fields. The remaining keys can have the following location_json inside them: {"country_code":"us","state":"California","county":"San Francisco","city":"San Francisco"}.

5. **user_location**: It can have a "location_json" as described above or an empty JSON {}. This field uses the "location" profile meta-data of a Twitter user and represents the user declared location in the text format. We resolve the text to a location. 

6. **geo**: represents the "geo" field provided by Twitter. We resolve the provided latitude and longitude values to locations. It can have a "location_json" as described above or an empty JSON {}.

7. **tweet_locations**: This field can have an array of "location_json" as described above [location_json1, location_json2] or an empty array []. This field uses the tweet content to find toponyms. 

8. **place**: It can have a "location_json" described above or an empty JSON {}. It represents the Twitter-provided "place" field.

## Solução Técnica

### Resumo da Solução

Foi implementada uma solução para a leitura dos registros de tweets a partir dos arquivos de dados providos pelo trabalho GeoCoV19, selecionando registros que fazem referência ao Brasil. 

Os dados foram disponibilizados em arquivos diários compactados no formato "zip", totalizando 90 arquivos. A solução realiza a descompactação de cada um desses arquivos (que possuem formato JSON) selecionando os registros desejados e criando um novo arquivo JSON com esses registros. 

Os arquivos disponibilizados possuem informações de geolocalização dos tweets (citadas no item anterior) e também um atributo informando quais destas localizações fornecidas é a mais precisa (atributo "geo_source"). Através deste atributo selecionamos os registros desejados (contendo a coluna country_code = "br") durante a leitura dos arquivos JSON.

Os registros de tweets brasileiros selecionados são armazenados em uma banco de dados MongoDB. Em um segundo momento, realizamos o "hydrate" desses tweets a partir dos seus ids utilizando a ferramenta Twarc. Por último, realizamos a tradução dos textos para inglês, com a utilização da ferramenta Googletrans e calculamos seus escores de sentimentos, utilizando o Vader Sentiment, ferramenta desenvolvida especialmente para a utilização em textos de redes sociais.

### Importações

In [1]:
# Importando módulos internos
import geocov19_functions_db as fdb
import geocov19_functions_tweets as ftweets
import geocov19_functions_text as ftext

# Importando módulos externos
import pandas as pd

[nltk_data] Downloading package stopwords to /home/mario/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mario/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Variáveis

In [12]:
# Diretório base
#home = '/home/mario/Arquivos'
#dados = '/media/mario/Dados1/Arquivos'
test = '/media/mario/Dados2/Arquivos'

# Diretório dos arquivos zip 
zip_dir = test + '/input/geo'

# Diretório de descompactação dos arquivos zip (arquivos json)
json_dir = test + '/output/json'

# Diretório dos arquivos json com geolocalizações brasileiras processados a partir dos zips descompactados
geo_dir = test + '/output/geo'

# Diretório dos arquivos de ids
ids_dir = test + '/output/ids'

# Diretório de downloads realizados pelo Twarc
downloads_dir = test + '/downloads'

# Diretório de tweets com textos
tweets_dir = test + '/output/tweets'

### Conexão ao BD

In [2]:
# Criando estrutura do banco de dados
from pymongo import MongoClient

# Conexão com o servidor do MongoDB
client = MongoClient('localhost', 27017)

# Conexão com a base de dados do mongoDB
db = client.SpedDB

# Coleção onde serão inseridos os dados
#collection = db.tweets_brasil_test
collection = db.tweets_brasils

#print(collection.count_documents({}))

### Realização da leitura dos arquivos zip

In [4]:
# Realizando a extração dos arquivos zip
ftweets.read_files(zip_dir, json_dir, geo_dir, ids_dir, 'br')

Extraindo arquivos...
-> Arquivo ''geo_2020-02-01.zip' já extraído anteriormente (1/2)
-> Arquivo ''geo_2020-02-02.zip' já extraído anteriormente (2/2)
2 arquivo(s) extraídos(s)
Processando arquivo(s) extraído(s)...
Lendo arquivo 'geo_2020-02-01.json com 666573 linhas
Tweets válidos encontrados: 954
-> Gerando arquivo json 'geo_2020-02-01.json
-> Gerando arquivo com ids '/media/mario/Dados2/Arquivos/output/ids/geo_2020-02-01.json_ids.csv
Tweets válidos até o momento: 954
Lendo arquivo 'geo_2020-02-02.json com 1462153 linhas
Tweets válidos encontrados: 10798
-> Gerando arquivo json 'geo_2020-02-02.json
-> Gerando arquivo com ids '/media/mario/Dados2/Arquivos/output/ids/geo_2020-02-02.json_ids.csv
Tweets válidos até o momento: 11752


## Salvando informações no BD

In [6]:
# Lendo arquivos json com registros brasileiros
json_files = ftweets.list_files(geo_dir, ".json")

In [7]:
# Criando banco de dados de tweets com as geolocalizações brasileiras
fdb.create_db(collection, json_files)

Criando tweets do arquivo /media/mario/Dados2/Arquivos/output/geo/geo_2020-02-01.json
Criando tweets do arquivo /media/mario/Dados2/Arquivos/output/geo/geo_2020-02-02.json


In [2]:
# Retornando quantidade de tweets inseridos
collection.count_documents({})

6186812

In [3]:
# Retornando o primeiro tweet
collection.find_one({})

{'_id': ObjectId('5f501410da596ab0c7238d1c'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'user_id': '1199299942797500423',
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'text': 'Director General of Health Services Dr. Anil Jasinghe confirmed the Chinese woman admitted at IDH has recovered and can be discharged by tomorrow #Lka #SriLanka #coronavirus',
 'score': 0.1027,
 'lang': 'en',
 'period': '2020_02_01'}

## Populando textos dos tweets

A partir dos arquivos de ids gerados no procedimento de leitura dos arquivos zips, foi realizado o "hydrate" dos tweets, a partir desses ids, utilizando a ferramento Twarc.

### Atualização do banco de dados

Realizando a atualização no banco de dados com os textos dos tweets retornados pelo Twarc

In [10]:
# Criando index para o atributo tweet_id
fdb.create_index(collection, 'tweet_id')

In [5]:
# Listando arquivos json gerados pelo Twarc
twarc_files = ftweets.list_files(downloads_dir, '.json')

# Atualizando banco de dados com os atributos Text e Lang a partir dos arquivos Json gerados pelo Twarc
fdb.update_tweets_db(collection, twarc_files)

Atualizando tweets do arquivo /media/mario/Dados2/Arquivos/downloads/tweets_2020-02-01.json
Atualizando tweets do arquivo /media/mario/Dados2/Arquivos/downloads/tweets_2020-02-02.json


In [6]:
# Criando index para outros atributos alvos de selects
fdb.create_index(collection, 'state')
fdb.create_index(collection, 'city')
fdb.create_index(collection, 'lang')
fdb.create_index(collection, 'geo_source')
fdb.create_index(collection, 'created_at')
fdb.create_index(collection, 'polarity')

In [7]:
# Retornando quantidade de tweets com textos não nulos
collection.count_documents({'text': {'$ne':None}})

8494

In [14]:
# Retornando o primeiro tweet
collection.find_one({})

{'_id': ObjectId('5f501410da596ab0c7238d1c'),
 'tweet_id': 1223563091025301507,
 'created_at': datetime.datetime(2020, 2, 1, 11, 5, 48),
 'user_id': '1199299942797500423',
 'geo_source': 'tweet_text',
 'state': 'Rio de Janeiro',
 'city': 'Rio de Janeiro',
 'text': 'Director General of Health Services Dr. Anil Jasinghe confirmed the Chinese woman admitted at IDH has recovered and can be discharged by tomorrow #Lka #SriLanka #coronavirus',
 'score': 0.1027,
 'lang': 'en',
 'period': '2020_02_01'}

## Análise de Sentimentos

Função para tradução do texto para inglês utilizando a ferramenta "Googletrans".

In [3]:
from googletrans import Translator

translator = Translator()

def translate_tweet_text(tweet):
    
    translated = ''
        
    text = tweet['text']   
    from_lang = tweet['lang']
    
    # Verificando se o texto do tweet já está escrito na língua inglesa
    if (from_lang == 'en') or (from_lang == 'en-US'):
        translated = text
    else:
        translated = translator.translate(text, dest='en').text

    return translated

Função para o cálculo do score de sentimento de um texto utilizando a ferramenta VaderSentiment.

In [5]:
# Ferramenta VaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def calculate_scores(tweets):

    for tweet in tweets:
                
        # Traduzindo o texto para o inglês
        translated_text = translate_tweet_text(tweet)
        
        # Efetuando o cálculo do score de sentimento para o texto
        vs = analyzer.polarity_scores(translated_text)
        
        # Recuperando o score e adicionando em uma nova célula do dataframe
        collection.update_one({"tweet_id": tweet['tweet_id']}, {'$set':{"score": vs['compound']}})      

Funções para o cálculo da intensidade da polaridade de um texto utilizando a ferramenta SenticNet.

In [6]:
import sys

# Ferramenta SenticNet
def calculate_polarities(tweets):
    
    for tweet in tweets:        
        try:       
            # Traduzindo o texto para o inglês
            translated_text = translate_tweet_text(tweet)

            # Realizando a limpeza da frase
            words = ftext.process_tweet(translated_text, 'english')

            # Efetuando o cálculo do score de sentimento para o texto
            polarity = calculate_words_polarities(words)

            # Recuperando o score e adicionando em uma nova célula do dataframe
            collection.update_one({"tweet_id": tweet['tweet_id']},{'$set':{"polarity": polarity, "senticnet_processed": True}})  
        
        except:
            #print("Skipping id "+str(tweet['tweet_id']+" Erro: "+e))
            e = sys.exc_info()[1]
            print ("Unexpected error:", e)
              

In [7]:
from senticnet.senticnet import SenticNet

def calculate_words_polarities(words):
    
    if(len(words) > 0):
        min_words_count = len(words)/2
        not_found_words_count = 0
        total_polarity_value = 0
        language_code = 'en'

        for word in words:
            word = word.lower()
            try:
                #polarity_value = verify_polarity(word)
                sn = SenticNet('en')
                polarity_str = sn.polarity_intense(word)
                polarity_value = float(polarity_str)      
                total_polarity_value = total_polarity_value + polarity_value
            except KeyError:
                not_found_words_count = not_found_words_count + 1

        # O cálculo será considerado somente se ao menos metade das palavras forem encontradas na base de conhecimento do SenticNet
        if not_found_words_count > min_words_count:
            return None
        else:
            return total_polarity_value / len(words)
    else:
        return None

Recuperando tweets da base
- Tweets que tenham geolocalização restritas a "place" e "user_location" (as localizações "tweet_texts" foram excluídas)
- Tweets que tenham scores (Vader) e polarities (SenticNet) calculados

In [14]:
# Recuperando tweets da base
tweets = collection.find({'text':{'$ne':None}, "senticnet_processed": {'$eq':None}, 'score': {'$ne':None}, '$or':[{'geo_source':'place'}, {'geo_source':'user_location'}]}, {'tweet_id':1,'lang':1,'text':1,'_id': 0})  

Realizando o cálculo do score de sentimento com o Vader para todos os tweets:

In [110]:
calculate_scores(tweets)

Realizando o cálculo da intensidade da polaridade com o SenticNet para todos os tweets

In [20]:
calculate_polarities(tweets)

## Fontes

(1) GeoCoV19: A Dataset of Hundreds of Millions ofMultilingual COVID-19 Tweets with Location Information<br>

(2) Paper Info, Statics and Downloads - https://crisisnlp.qcri.org/covid19<br>

(3) Pyhton Documentation - https://docs.python.org/3/ 

(4) Muhammad Imran, Prasenjit Mitra, Carlos Castillo: Twitter as a Lifeline: Human-annotated Twitter Corpora for NLP of Crisis-related Messages. In Proceedings of the 10th Language Resources and Evaluation Conference (LREC), pp. 1638-1643. May 2016, Portorož, Slovenia.

(5) How to Generate API Key, Consumer Token, Access Key for Twitter OAuth - https://themepacific.com/how-to-generate-api-key-consumer-token-access-key-for-twitter-oauth/994/

(6) Iterative JSON parser with a standard Python iterator interface - https://pypi.org/project/ijson/

(7) Make working with large DataFrames easier, at least for your memory - https://towardsdatascience.com/make-working-with-large-dataframes-easier-at-least-for-your-memory-6f52b5f4b5c4

(8) Twitter API Documentation - Tweet Location Metadata - https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/geo-objects

(9) Twarc - https://github.com/DocNow/twarc

(10) Map Polygon - https://www.keene.edu/campus/maps/tool/

(11) MongoBD Documentation - https://docs.mongodb.com/manual/reference/command/count/

(12) Free Google Translator API for Pyhton - https://pypi.org/project/googletrans/